# 📘 1枚画像 → 3Dメッシュ（SPAR3D）Colab ノートブック（安定版 v1）

- 表示の安定化（`/files` → Blob 経由）
- `transparent-background>=1.3.4` / `run.py` Hotfix を自動適用

## Step 0 — GPU 確認

In [ ]:
!nvidia-smi || echo 'No GPU?'

## Step 1 — 依存のセットアップとリポジトリ取得

In [ ]:
%env NO_ALBUMENTATIONS_UPDATE=1
!pip -q install -U pip wheel "setuptools==69.5.1" ninja cmake

!git clone -q https://github.com/Stability-AI/stable-point-aware-3d.git
%cd stable-point-aware-3d

!pip -q install -r requirements.txt
!pip -q install -U "transparent-background>=1.3.4"

# （任意）高品質リメッシュ
# !pip -q install gpytoolbox pynanoinstantmeshes==0.0.3
# !pip -q install -r requirements-remesh.txt

## Step 2 — Hugging Face にログイン（トークン貼付）

In [ ]:
from huggingface_hub import login
login()

## Step 3 — 入力画像アップロード → 512×512 に整形

In [ ]:
from google.colab import files
from PIL import Image
import os

uploaded = files.upload()
in_name = list(uploaded.keys())[0]

img = Image.open(in_name).convert("RGB")
w, h = img.size
size = max(w, h)
canvas = Image.new("RGB", (size, size), (255, 255, 255))
canvas.paste(img, ((size - w)//2, (size - h)//2))

img512 = canvas.resize((512, 512), Image.LANCZOS)
os.makedirs("inputs", exist_ok=True)
prep_path = "inputs/input_512.png"
img512.save(prep_path)
print("Prepared:", prep_path, "->", img512.size)

## Step 4 — 安定動作用 Hotfix（`run.py` に小修正を注入）

In [ ]:
from pathlib import Path
p = Path("run.py")
s = p.read_text()

inject = """
# --- Hotfix: ensure defaults when remeshing backends are missing ---
if not hasattr(args, "reduction_count_type"):
    args.reduction_count_type = "keep"
if not hasattr(args, "target_count"):
    args.target_count = 2000
# --- End hotfix ---
"""

if "Hotfix: ensure defaults" not in s:
    s = s.replace("args = parser.parse_args()", "args = parser.parse_args()" + inject)
    p.write_text(s)
    print("run.py に Hotfix を適用しました。")
else:
    print("Hotfix は既に適用済みです。")

## Step 5 — 推論（.glb 生成）

In [ ]:
# %env SPAR3D_LOW_VRAM=1  # 省メモリが必要なら有効化
!mkdir -p outputs
!python run.py inputs/input_512.png --output-dir outputs --texture-resolution 1024 --remesh_option none

## Step 6 — 生成ファイルの検出

In [ ]:
import glob, os
candidates = sorted(glob.glob("outputs/**/*.glb", recursive=True), key=os.path.getmtime)
assert candidates, "GLBが見つかりません。ログを確認してください。"
glb_path = candidates[-1]
abs_glb = os.path.abspath(glb_path)
print("GLB:", glb_path)
print("ABS:", abs_glb)

## Step 7 — プレビュー（`<model-viewer>`：Blob経由で読込）

In [ ]:
from IPython.display import HTML, display
html = f"""
<div id='mv_wrap' style='width:100%;height:520px;background:#f4f4f4;border-radius:8px;'></div>
<script type="module">
  import "https://unpkg.com/@google/model-viewer/dist/model-viewer.min.js";
  const wrap = document.getElementById('mv_wrap');
  const mv = document.createElement('model-viewer');
  mv.setAttribute('camera-controls', '');
  mv.setAttribute('auto-rotate', '');
  mv.setAttribute('shadow-intensity', '1');
  mv.style.width = '100%';
  mv.style.height = '100%';
  wrap.appendChild(mv);

  async function loadGLB() {{
    const r = await fetch("/files{abs_glb}");
    if (!r.ok) throw new Error('fetch failed: ' + r.status);
    const buf = await r.arrayBuffer();
    const url = URL.createObjectURL(new Blob([buf], {{type: "model/gltf-binary"}}));
    mv.src = url;
  }}
  loadGLB().catch(e => {{
    wrap.innerHTML = '<div style="padding:16px;color:#b00">model-viewer 読み込みに失敗: '+e+'</div>';
    console.error(e);
  }});
</script>
<p style="margin:8px 0 0 0;">Path: <code>{abs_glb}</code></p>
"""
display(HTML(html))

## Step 8 — メッシュとテクスチャの統計

In [ ]:
import struct, json, base64, io, os
from PIL import Image
from IPython.display import display, Markdown, HTML

def parse_glb(glb_path):
    with open(glb_path, "rb") as f:
        data = f.read()
    assert data[:4] == b'glTF', "Not a GLB"
    version = struct.unpack_from("<I", data, 4)[0]
    assert version == 2, f"glTF version: {version}"
    json_length = struct.unpack_from("<I", data, 12)[0]
    json_start = 20
    json_end   = json_start + json_length
    gltf = json.loads(data[json_start:json_end].decode("utf-8"))
    bin_chunk = None
    if json_end + 8 <= len(data):
        bin_length = struct.unpack_from("<I", data, json_end)[0]
        bin_type   = struct.unpack_from("<I", data, json_end + 4)[0]
        if bin_type == 0x004E4942:
            bin_start = json_end + 8
            bin_chunk = data[bin_start:bin_start+bin_length]
    return gltf, bin_chunk

def accessor_count(gltf, idx): return gltf["accessors"][idx]["count"]

def analyze_mesh(gltf):
    total_v = 0; total_i = 0; prims = 0
    bbmin = [None]*3; bbmax = [None]*3
    for m in gltf.get("meshes", []):
        for p in m.get("primitives", []):
            prims += 1
            attrs = p.get("attributes", {})
            if "POSITION" in attrs:
                c = accessor_count(gltf, attrs["POSITION"]); total_v += c
                acc = gltf["accessors"][attrs["POSITION"]]
                if "min" in acc and "max" in acc:
                    for i in range(3):
                        mn, mx = acc["min"][i], acc["max"][i]
                        bbmin[i] = mn if bbmin[i] is None else min(bbmin[i], mn)
                        bbmax[i] = mx if bbmax[i] is None else max(bbmax[i], mx)
            if "indices" in p:
                total_i += accessor_count(gltf, p["indices"])
    tri = total_i//3 if total_i else 0
    bbsize = None
    if all(v is not None for v in bbmin + bbmax):
        bbsize = [bbmax[i]-bbmin[i] for i in range(3)]
    return total_v, tri, prims, bbmin, bbmax, bbsize

def material_texture_roles(gltf):
    roles = []
    tex_to_img = {i:t.get("source") for i,t in enumerate(gltf.get("textures", []))}
    for midx, m in enumerate(gltf.get("materials", [])):
        r = {"material": midx, "name": m.get("name")}
        pbr = m.get("pbrMetallicRoughness", {})
        if "baseColorTexture" in pbr: r["baseColorTexture_image"] = tex_to_img.get(pbr["baseColorTexture"]["index"])
        if "metallicRoughnessTexture" in pbr: r["metallicRoughnessTexture_image"] = tex_to_img.get(pbr["metallicRoughnessTexture"]["index"])
        if "normalTexture" in m: r["normalTexture_image"] = tex_to_img.get(m["normalTexture"]["index"])
        if "occlusionTexture" in m: r["occlusionTexture_image"] = tex_to_img.get(m["occlusionTexture"]["index"])
        if "emissiveTexture" in m: r["emissiveTexture_image"] = tex_to_img.get(m["emissiveTexture"]["index"])
        roles.append(r)
    return roles

def get_image_bytes(gltf, bin_chunk, img):
    if "uri" in img and img["uri"].startswith("data:"):
        b64 = img["uri"].split(";base64,",1)[-1]
        return base64.b64decode(b64)
    if "bufferView" in img and bin_chunk is not None:
        bv = gltf["bufferViews"][img["bufferView"]]
        offset = bv.get("byteOffset", 0)
        size   = bv["byteLength"]
        return bin_chunk[offset:offset+size]
    return None

gltf, bin_chunk = parse_glb(abs_glb)
v, f, prims, bbmin, bbmax, bbsize = analyze_mesh(gltf)
roles = material_texture_roles(gltf)

md = f"""
**Mesh summary**

- meshes: `{len(gltf.get('meshes', []))}` / primitives: `{prims}`
- vertices: `{v}` / triangles: `{f}`
- bbox min: `{bbmin}` / max: `{bbmax}` / size (xyz): `{bbsize}`

**Materials**: `{len(gltf.get('materials', []))}`
**Textures**: `{len(gltf.get('textures', []))}` / **Images**: `{len(gltf.get('images', []))}`
"""
display(Markdown(md))

def html_table(rows, headers):
    th = "".join(f"<th>{h}</th>" for h in headers)
    trs = []
    for r in rows:
        tds = "".join(f"<td>{r.get(h,'')}</td>" for h in headers)
        trs.append(f"<tr>{tds}</tr>")
    return f"<table style='border-collapse:collapse'><thead><tr>{th}</tr></thead><tbody>{''.join(trs)}</tbody></table>"

roles_headers = ["material","name","baseColorTexture_image","metallicRoughnessTexture_image","normalTexture_image","occlusionTexture_image","emissiveTexture_image"]
display(HTML("<h4>Per-material texture roles</h4>" + html_table(roles, roles_headers)))

img_rows = []
for i, img in enumerate(gltf.get("images", [])):
    raw = get_image_bytes(gltf, bin_chunk, img)
    w = h = None
    if raw:
        try:
            im = Image.open(io.BytesIO(raw))
            w, h = im.size
        except Exception:
            pass
    img_rows.append({"image_index": i, "mimeType": img.get("mimeType"), "width": w, "height": h})
display(HTML("<h4>Embedded images</h4>" + html_table(img_rows, ["image_index","mimeType","width","height"])))

## Step 9 — 生成物のダウンロード

In [ ]:
from google.colab import files
files.download(abs_glb)